In [1]:
from sentence_transformers import SentenceTransformer, SentencesDataset, util, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
#instalação do Ulysses Fetcher, para buscar os datasets no Github do projeto
!python -m pip install "git+https://github.com/ulysses-camara/ulysses-fetcher@v0.1.1-beta"
import buscador
buscador.download_model(
    task_name="sentence_similarity",
    model_name="distil_sbert_br_ctimproved_12_epochs_v1",
)

  Cloning https://github.com/ulysses-camara/ulysses-fetcher (to revision v0.1.1-beta) to c:\users\lucas\appdata\local\temp\pip-req-build-e8junje7
  Resolved https://github.com/ulysses-camara/ulysses-fetcher to commit 933107853f4794beb2ffbd99ec4c25437e569aea
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/ulysses-camara/ulysses-fetcher 'C:\Users\lucas\AppData\Local\Temp\pip-req-build-e8junje7'
  Running command git checkout -q 933107853f4794beb2ffbd99ec4c25437e569aea
You should consider upgrading via the 'c:\Users\lucas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


True

In [3]:
model_name = 'distil_sbert_br_tuned'
train_batch_size = 16
num_epochs = 5
model_save_path = '.\distil_sbert_br_tuned'

In [4]:
model = SentenceTransformer(
    "distil_sbert_br_ctimproved_12_epochs_v1",
    device="cpu",
)


In [8]:
df = pd.read_csv("usp_feedback_separado.csv")

In [11]:
df.drop(columns=['score', 'score_normalized', 'score_final', 'tipo', 'Unnamed: 0'], inplace=True)

In [13]:
df.head()

,query,name,class
0,Solicito projeto de lei para dispensar Guia de...,PL 3650/2021,r
1,Solicito projeto de lei para dispensar Guia de...,2896/2022,r
2,Solicito projeto de lei para dispensar Guia de...,PL 8104/2017,i
3,Solicito projeto de lei para dispensar Guia de...,PL 1689/2019,i
4,Solicito projeto de lei para dispensar Guia de...,PL 3514/2008,i


In [14]:
replacement_map = {
    "r" : 1.0,
    "i" : 0.0
}
df['class'] = df['class'].replace(replacement_map)

In [17]:
for i in range(len(df)):
    if df["class"][i] == 'pr':
        df.drop(i, inplace=True)

In [18]:
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [19]:
df

,level_0,query,name,class
0,0,Solicito projeto de lei para dispensar Guia de...,PL 3650/2021,1.0
1,1,Solicito projeto de lei para dispensar Guia de...,2896/2022,1.0
2,2,Solicito projeto de lei para dispensar Guia de...,PL 8104/2017,0.0
3,3,Solicito projeto de lei para dispensar Guia de...,PL 1689/2019,0.0
4,4,Solicito projeto de lei para dispensar Guia de...,PL 3514/2008,0.0
...,...,...,...,...
13940,13940,SEGURO-DEFESO DEPENDENTES DE PESCADORES ARTESA...,3636/2005,0.0
13941,13941,SEGURO-DEFESO DEPENDENTES DE PESCADORES ARTESA...,17795/2013,0.0
13942,13942,SEGURO-DEFESO DEPENDENTES DE PESCADORES ARTESA...,908/2007,0.0
13943,13943,SEGURO-DEFESO DEPENDENTES DE PESCADORES ARTESA...,5306/2005,0.0


In [20]:
df_corpus = pd.read_csv("usp_corpus_20220714.csv")

In [21]:
df_corpus.head()

,code,sig_tipo,name,txt_ementa,em_tramitacao,situacao,text,text_preprocessed
0,2307087,INC,INC 1255/2021,Requer ao Poder Executivo federal o aumento do...,Sim,Aguardando Remessa ao Arquivo,"INDICAÇÃO Nº , DE 2021\n(Do Sr. ZÉ SILVA)...","['indicaca', '2021', 'sr', 'ze', 'silv', 'requ..."
1,2306960,PDL,PDL 990/2021,Susta os efeitos de dispositivo do Decreto nº ...,Sim,Aguardando Despacho do Presidente da Câmara do...,"PROJETO DE DECRETO LEGISLATIVO Nº , DE...","['projet', 'decret', 'legislativ', '2021', 'sr..."
2,2306615,INC,INC 1234/2021,Sugere ao Ministério da Saúde a instalação de ...,Sim,Aguardando Resposta,"INDICAÇÃO No , DE 2021 \n(Da Se...","['indicaca', '2021', 'senhor', 'perpetu', 'alm..."
3,2306617,INC,INC 1236/2021,Sugere ao Ministério da Saúde a instalação de ...,Sim,Aguardando Resposta,"INDICAÇÃO No , DE 2021 \n(Da Se...","['indicaca', '2021', 'senhor', 'perpetu', 'alm..."
4,2306573,INC,INC 1230/2021,Sugere a aprovação de Convênio no Conselho Nac...,Sim,Aguardando Resposta,CÂMARA DOS DEPUTADOS\n Deputa...,"['camar', 'deputad', 'deputad', 'juli', 'lop',..."


In [22]:

train_samples = []
dev_samples = []
test_samples = []
for i in range(len(df)):
    if not (df_corpus.loc[df_corpus["name"] == df["name"][i]].empty):
        inp_example = InputExample(texts = [str(df["query"][i]).strip() , str(df_corpus.loc[df_corpus["name"] == df["name"][i]]["text"]).strip()], label= float(df["class"][i]))
        if i % 3 == 0:
            train_samples.append(inp_example)
        if i % 2 == 0: 
            test_samples.append(inp_example)
        if i % 7 == 0:
            dev_samples.append(inp_example)

In [23]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

In [24]:
train_loss = losses.CosineSimilarityLoss(model)

In [25]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

In [26]:
import math
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [27]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


c:\Users\lucas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]

In [28]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.49387786761978264